In [113]:
import pandas as pd
import numpy as np
df = pd.read_csv('all_data.csv', delimiter=';')
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# drop Name and SSN columns
df = df.drop(['Name', 'SSN'], axis=1)

C:\Users\timow\AppData\Local\Temp\ipykernel_9056\3027398730.py:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('all_data.csv', delimiter=';')


In [114]:
# Order entries by cutomer and month, and then reset index
month_to_num = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 
                'June': 6, 'July': 7, 'August': 8, 'September': 9, 
                'October': 10, 'November': 11, 'December': 12}

df["temp_Month"] = df["Month"].map(month_to_num)
df.sort_values(["Customer_ID", "temp_Month"], inplace=True)
df.drop("temp_Month", axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

print(df.head(12))

         ID Customer_ID      Month  Age Occupation Annual_Income  \
0   0x1628a  CUS_0x1000    January   17     Lawyer      30625,94   
1   0x1628b  CUS_0x1000   February   17     Lawyer      30625,94   
2   0x1628c  CUS_0x1000      March  17_     Lawyer     30625.94_   
3   0x1628d  CUS_0x1000      April   17     Lawyer      30625,94   
4   0x1628e  CUS_0x1000        May   17     Lawyer      30625,94   
5   0x1628f  CUS_0x1000       June   18     Lawyer      30625,94   
6   0x16290  CUS_0x1000       July   18     Lawyer      30625,94   
7   0x16291  CUS_0x1000     August   18     Lawyer      30625,94   
8   0x16292  CUS_0x1000  September   18     Lawyer      17188404   
9   0x16293  CUS_0x1000    October   18     Lawyer      30625,94   
10  0x16294  CUS_0x1000   November   18     Lawyer      30625,94   
11  0x16295  CUS_0x1000   December   18     Lawyer      30625,94   

   Monthly_Inhand_Salary  Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  \
0            2706,161667            

In [108]:
# In every iteration, the loop will replace all _______ with a value of either the previous or next month. 
# It seems that there are at most 3 consecutive months with _______ in Occupation, so 3 loops is enough. 
# If we want to make the code work for more than 3 consecutive months, we can add more loops.
for _ in range(3):
    for i in df.index:
        if df.loc[i, 'Occupation'] == '_______':
            if df.loc[i+1, "Customer_ID"] == df.loc[i, "Customer_ID"] and df.loc[i+1, 'Occupation'] != '_______':
                df.loc[i, 'Occupation'] = df.loc[i+1, 'Occupation']
            elif df.loc[i-1, "Customer_ID"] == df.loc[i, "Customer_ID"] and df.loc[i-1, 'Occupation'] != '_______':
                df.loc[i, 'Occupation'] = df.loc[i-1, 'Occupation']

# print entries with ______ in Occupation
df[df['Occupation'] == '_______']

,ID,Customer_ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score


In [116]:
# Find out which special characters are cluttering the data
df['test_Annual_Income'] = df['Annual_Income'].str.replace('\d+', '', regex=True)
print(df["test_Annual_Income"].value_counts())
df.drop("test_Annual_Income", axis=1, inplace=True)

# Replace all special characters with the correct ones
df['Annual_Income'] = df['Annual_Income'].str.replace('.', ',')
df['Annual_Income'] = df['Annual_Income'].str.replace('_', '')

# Check if you got all of them
df['test_Annual_Income'] = df['Annual_Income'].str.replace('\d+', '', regex=True)
print(df["test_Annual_Income"].value_counts())
df.drop("test_Annual_Income", axis=1, inplace=True)

test_Annual_Income
,    146373
       3627
Name: count, dtype: int64
Annual_Income
109945,32             24
32543,38              24
9141,63               24
17816,75              24
36585,12              24
40341,16              24
17273,83              24
22434,16              24
20867,67              24
95596,35              23
72524,2               23
33029,66              23
35952,25              12
34094,94              12
29423,32              12
63839,46              12
62030,64              12
72559,36              12
39126,01              12
173650,68             12
17621,705             12
44438,41              12
16276,325             12
120922,26             12
142496,12             12
16532,33              12
39206,98              12
41398,44              12
59505,12              12
33409,33              12
21370,185             12
15081,42              12
112815,48             12
19848,365             12
133439,76             12
20945,23              12
65804,04         